In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np
from sklearn.metrics import classification_report

# --- Data Loading and Preprocessing ---
import tensorflow as tf
import os
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Set paths to your dataset directories
dataset_dir = r"D:\Major Project\Final Proper\Final_eyes_data"  # Parent directory containing 'closed eyes' and 'open eyes' folders

# Define image size and batch size
target_img_size = (224, 224)  # Update based on model needs
batch_size = 32

# Load dataset (automatically labels based on folder names)
dataset = image_dataset_from_directory(
    dataset_dir,
    labels='inferred',         # Infer labels from folder names
    label_mode='binary',       # Binary classification (open vs. closed)
    batch_size=batch_size,
    image_size=target_img_size,
    shuffle=True,
    color_mode='grayscale'     # Convert images to grayscale if necessary
)

# Split dataset into training and testing (80% train, 20% test)
train_size = int(0.8 * len(dataset))
train_ds = dataset.take(train_size)
test_ds = dataset.skip(train_size)

# Normalize pixel values (scale to [0,1])
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Check dataset
for image_batch, label_batch in train_ds.take(1):
    print(f"Image batch shape: {image_batch.shape}")  # Expected: (batch_size, 224, 224, 1)
    print(f"Label batch shape: {label_batch.shape}")  # Expected: (batch_size,)

Found 12000 files belonging to 2 classes.
Image batch shape: (32, 224, 224, 1)
Label batch shape: (32, 1)


In [18]:
@tf.keras.utils.register_keras_serializable(package="Custom",name="margin_loss")
def margin_loss(y_true, y_pred):
    m_plus = 0.9
    m_minus = 0.1
    lambda_val = 0.5
    L = y_true * tf.square(tf.maximum(0., m_plus - y_pred)) + \
        lambda_val * (1 - y_true) * tf.square(tf.maximum(0., y_pred - m_minus))
    return tf.reduce_mean(tf.reduce_sum(L, axis=1))
@tf.keras.utils.register_keras_serializable(package="Custom",name="squash")
def squash(vectors, axis=-1):
    s_squared_norm = tf.reduce_sum(tf.square(vectors), axis=axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm)
    return scale * vectors / tf.sqrt(s_squared_norm + tf.keras.backend.epsilon())
@tf.keras.utils.register_keras_serializable(package="Custom",name="PrimaryCap")
def PrimaryCap(inputs, dim_capsule, n_channels, kernel_size, strides, padding):
    conv = layers.Conv2D(filters=dim_capsule * n_channels,
                         kernel_size=kernel_size,
                         strides=strides,
                         padding=padding,
                         activation='relu')(inputs)
    outputs = layers.Reshape(target_shape=[-1, dim_capsule])(conv)
    return layers.Lambda(squash)(outputs)
@tf.keras.utils.register_keras_serializable(package="Custom")
class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsule, dim_capsule, routings=2, **kwargs):
        """
        Simplified Capsule Layer with reduced routing iterations.
        """
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings

    def build(self, input_shape):
        self.input_num_capsule = input_shape[1]
        self.input_dim_capsule = input_shape[2]
        # Weight matrix: shape [input_num_capsule, num_capsule, input_dim_capsule, dim_capsule]
        self.W = self.add_weight(
            shape=[self.input_num_capsule, self.num_capsule, self.input_dim_capsule, self.dim_capsule],
            initializer='glorot_uniform',
            trainable=True,
            name='W'
        )
        super(CapsuleLayer, self).build(input_shape)

    def call(self, inputs):
        u_hat = tf.einsum('bij,imjn->bimn', inputs, self.W)
        b = tf.zeros(shape=[tf.shape(inputs)[0], self.input_num_capsule, self.num_capsule])
        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=2)
            c_expanded = tf.expand_dims(c, -1)
            s = tf.reduce_sum(c_expanded * u_hat, axis=1)
            v = squash(s)
            if i < self.routings - 1:
                v_expanded = tf.expand_dims(v, 1)
                b += tf.reduce_sum(u_hat * v_expanded, axis=-1)
        return v

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)
@tf.keras.utils.register_keras_serializable(package="Custom",name="capsule_length")
def capsule_length(x):
    return tf.sqrt(tf.reduce_sum(tf.square(x), axis=-1))

# Explicit output shape: if input is (batch, n_caps, dim_caps), output should be (batch, n_caps).
capsule_length_output_shape = lambda input_shape: (input_shape[0], input_shape[1])

from tensorflow.keras import layers, models

# def CapsNet(input_shape, n_class, routings=1):  # Reduce routings from 3 → 1
#     inputs = layers.Input(shape=input_shape)

#     # Reduce filters to avoid overfitting
#     conv1 = layers.Conv2D(filters=128, kernel_size=5, strides=1, padding='same', activation='relu')(inputs)
#     conv2 = layers.Conv2D(filters=128, kernel_size=5, strides=2, padding='same', activation='relu')(conv1)
#     conv2 = layers.BatchNormalization()(conv2)
#     conv2 = layers.Dropout(0.3)(conv2)  # Dropout to prevent overfitting


#     # Reduce n_channels and dim_capsule to reduce parameters
#     primary_caps = PrimaryCap(conv2, dim_capsule=8, n_channels=16, kernel_size=5, strides=2, padding='valid')

#     # Reduce digit capsule dimensions
#     digit_caps = CapsuleLayer(num_capsule=n_class, dim_capsule=8, routings=routings, name='digit_caps')(primary_caps)

#     # Capsule length (Output)
#     out_caps = layers.Lambda(capsule_length, output_shape=capsule_length_output_shape)(digit_caps)

#     model = models.Model(inputs=inputs, outputs=out_caps)

#     return model

In [19]:
custom_objects = {
    'CapsuleLayer': CapsuleLayer,
    'PrimaryCap': PrimaryCap,
    'squash': squash,
    'capsule_length': capsule_length,
    'margin_loss': margin_loss,  # include if you're using a custom margin loss

}


In [20]:
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input

# Load the old model (trained on RGB)
old_model=tf.keras.models.load_model("drowsiness_capsnet_jupyter.keras",custom_objects=custom_objects,safe_mode=False)

In [21]:
old_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 224, 224, 128)       │           9,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 112, 112, 128)       │         409,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 112, 112, 128)       │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 112, 112, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 54, 54, 128)         │         409,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 46656, 8)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda (Lambda)                      │ (None, 46656, 8)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ digit_caps (CapsuleLayer)            │ (None, 2, 8)                │       5,971,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_1 (Lambda)                    │ (None, 2)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,603,074 (51.89 MB)

 Trainable params: 6,801,408 (25.95 MB)

 Non-trainable params: 256 (1.00 KB)

 Optimizer params: 6,801,410 (25.95 MB)

In [22]:
from tensorflow.keras.layers import Input, Conv2D

# Get model layers except the first one
new_input = Input(shape=(224, 224, 1), name="new_input_layer")

# --- Convert Grayscale to 3 Channels Using a 1x1 Conv2D Layer ---
converted_input = Conv2D(3, (1, 1), activation="linear", name="convert_to_rgb")(new_input)

# --- Connect the Converted Input to the Old Model ---
x = old_model.layers[1](converted_input)  # Start from the second layer of the old model
for layer in old_model.layers[2:]:  # Add all remaining layers
    x = layer(x)

# --- Create the New Model with Grayscale Input ---
new_model = Model(new_input, x, name="grayscale_to_rgb_model")

# Compile and use this new model
optimizer = tf.keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9, nesterov=True)
new_model.compile(optimizer=optimizer, loss=margin_loss, metrics=['accuracy'])

In [23]:
#input_shape = target_img_size + (1,)
# --- Build and Compile the Model ---
# model = CapsNet(input_shape=input_shape, n_class=2, routings=3)
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9, nesterov=True)
# model.compile(optimizer=optimizer, loss=margin_loss, metrics=['accuracy'])

new_model.summary()
# --- Learning Rate Scheduler Callback ---
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='loss', factor=0.5, patience=3, verbose=1, min_lr=1e-5
)
# --- Train the Model ---
new_model.fit(train_ds, epochs=10, callbacks=[lr_scheduler])


Model: "grayscale_to_rgb_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ new_input_layer (InputLayer)         │ (None, 224, 224, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ convert_to_rgb (Conv2D)              │ (None, 224, 224, 3)         │               6 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 224, 224, 128)       │           9,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 112, 112, 128)       │         409,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 112, 112, 128)       │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 112, 112, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 54, 54, 128)         │         409,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 46656, 8)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda (Lambda)                      │ (None, 46656, 8)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ digit_caps (CapsuleLayer)            │ (None, 2, 8)                │       5,971,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_1 (Lambda)                    │ (None, 2)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,801,670 (25.95 MB)

 Trainable params: 6,801,414 (25.95 MB)

 Non-trainable params: 256 (1.00 KB)

Epoch 1/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 3249s 11s/step - accuracy: 0.3866 - loss: 0.1336 - learning_rate: 5.0000e-04
Epoch 2/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 3273s 11s/step - accuracy: 0.4057 - loss: 0.0563 - learning_rate: 5.0000e-04
Epoch 3/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 3265s 11s/step - accuracy: 0.4369 - loss: 0.0476 - learning_rate: 5.0000e-04
Epoch 4/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 3278s 11s/step - accuracy: 0.4568 - loss: 0.0427 - learning_rate: 5.0000e-04
Epoch 5/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 3257s 11s/step - accuracy: 0.4469 - loss: 0.0406 - learning_rate: 5.0000e-04
Epoch 6/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 3249s 11s/step - accuracy: 0.4556 - loss: 0.0380 - learning_rate: 5.0000e-04
Epoch 7/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 3242s 11s/step - accuracy: 0.4570 - loss: 0.0356 - learning_rate: 5.0000e-04
Epoch 8/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 3708s 12s/step - accuracy: 0.4431 - loss: 0.0338 - learning_rate: 5.0000e-04
Epoch 9/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4664s 16s/step - accurac

In [26]:
# --- Evaluate and Print Classification Report ---
y_true = []
y_pred = []
for images, labels in test_ds:
    # Convert (32, 224, 224, 1) → (32, 224, 224, 3)
    images = tf.image.grayscale_to_rgb(images)
    
    preds = model.predict(images)  # Now the shape should be correct
    y_true.extend(np.argmax(labels, axis=1))
    y_pred.extend(np.argmax(preds, axis=1))

print(classification_report(y_true, y_pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━

D:\Major Project\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Major Project\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Major Project\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
for images, labels in test_ds:
    print(images.shape)  # Should be (batch_size, 224, 224, 3)
    break


In [25]:
model.save("drowsiness_capsnet_Eye_data.keras")

In [ ]:
custom_objects = {
    'CapsuleLayer': CapsuleLayer,
    'squash': squash,
    'PrimaryCap': PrimaryCap,
    'capsule_length': capsule_length,
    'margin_loss': margin_loss,  # include if you're using a custom margin loss

}
model_2=tf.keras.models.load_model('/content/drive/MyDrive/Main_Project/drowsiness_capsnet.keras',custom_objects=custom_objects,safe_mode=False)

In [ ]:
# --- Evaluate and Print Classification Report ---
y_true = []
y_pred = []
for images, labels in test_ds:
    preds = model_2.predict(images)
    y_true.extend(np.argmax(labels, axis=1))
    y_pred.extend(np.argmax(preds, axis=1))

print(classification_report(y_true, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred)
print(cm)
TN, FP, FN, TP = cm.ravel()

# Calculate sensitivity and specificity
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)

# Print sensitivity and specificity
print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc


y_pred_prob = model_2.predict(test_ds)  # (test_ds is the test dataset)
print(y_pred_prob)
print(y_pred_prob.shape)
y_pred_prob = y_pred_prob[:, 1]
# Flatten the predictions to match the shape of the true labels
y_pred_prob = y_pred_prob.flatten()
print(y_pred_prob)
# --- Get True Labels (y_test) ---
# Since `y_test` are one-hot encoded, we need to get the actual class labels.
y_test = np.concatenate([y for x, y in test_ds], axis=0)
y_test = np.argmax(y_test, axis=1)  # Convert one-hot encoding to class labels (0 or 1)

# --- Compute ROC Curve ---
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# --- Plot the ROC Curve ---
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  # Diagonal line (no discrimination)
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Assume y_test contains true class labels (0 or 1) and y_pred_prob contains predicted probabilities for class 1.
# You can generate binary predictions by applying a threshold (e.g., 0.5):


# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix Heatmap")
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

def calculate_error_rate(true_labels, predicted_labels):
    # Calculate accuracy first
    accuracy = accuracy_score(true_labels, predicted_labels)
    # Error rate is 1 minus accuracy
    error_rate = 1 - accuracy
    return error_rate



error_rate = calculate_error_rate(y_true,y_pred)
print(f"Error Rate: {error_rate:.2f}")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score

# Compute precision and recall for different thresholds
precision, recall, thresholds = precision_recall_curve(y_true, y_pred_prob)

# Calculate the average precision score
average_precision = average_precision_score(y_true, y_pred_prob)

# Plot the precision-recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='b', lw=2, label=f'Precision-Recall curve (AP = {average_precision:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='best')
plt.grid(True)
plt.show()
